In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
!nvidia-smi


In [ ]:
import glob
import cv2
import numpy as np
import tensorflow as tf
import math
from tensorflow.keras.layers import Conv2D, Input, Activation, Flatten, Dense, TimeDistributed, LSTM, Concatenate, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from network import ResNet18, KF
from dataloader import LoadData, DataPreprocessing

In [ ]:
#Hyper Parameters
batch_size = 8
lr = 0.00001
epochs = 300
input_shape1 = (5,224,224,3)
input_shape2 = (5,224,224,1)
data_path = 'data/'
data_name = 'TapData_P1_window5.npz'

In [ ]:
#Data Loader
print('Loading Data...')
#training data
data = np.load(data_path+'train/'+ data_name)
train_x = data['arr_0']
train_y = data['arr_1']
if train_x.shape[0] == train_y.shape[0]:
    print(str(train_x.shape[0]*train_x.shape[1]) + " training images loaded.")

#test data
data = np.load(data_path+'test/'+ data_name)
test_x = data['arr_0']
test_y = data['arr_1']
if test_x.shape[0] == test_y.shape[0]:
    print(str(test_x.shape[0]*train_x.shape[1]) + " test images loaded.")

output_shape = train_y.shape[1]

In [ ]:
#remove background, generate motion history
train_x1, train_x2 = DataPreprocessing(train_x, input_shape1, input_shape2) 
test_x1, test_x2 = DataPreprocessing(test_x, input_shape1, input_shape2)

In [ ]:
#Create data sequence
train_ds = LoadData(train_x1,train_x2,train_y,batch_size)
vali_ds = LoadData(test_x1,test_x2,test_y,batch_size)

In [ ]:
def network(input_shape1, input_shape2, output_shape, batch_size):
    
    resnet18 = ResNet18(weights=None, include_top=False)
    resnet18_gray = ResNet18(weights=None, include_top=False, input_shape=(224,224,1))

    inp1 = Input(input_shape1,batch_size)
    inp2 = Input(input_shape2,batch_size)
    
    x1 = TimeDistributed(resnet18)(inp1)
    x1 = TimeDistributed(Flatten())(x1)
    
    x2 = TimeDistributed(resnet18_gray)(inp2)
    x2 = TimeDistributed(Flatten())(x2)
    
    x = Concatenate()([x1,x2])
    x = TimeDistributed(Dense(64))(x)
    lstm_out = LSTM(128, stateful=True)(x)
    x = KF()(lstm_out)
    x = Add()([x,lstm_out])
    x = Dense(64)(x)
    
    out = Dense(output_shape, activation = 'sigmoid')(x)
  
    model = Model([inp1,inp2], out)
    return model

In [ ]:
model = network((5,224,224,3),(5,224,224,1), output_shape, batch_size)
opt = Adam(learning_rate = lr)
model.compile(loss='mse', optimizer=opt)
#model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
cp = ModelCheckpoint(filepath = 'model/PredConvLSTM-{epoch:03d}.h5',
                     save_weights_only = False,
                     #monitor = 'val_loss',
                     #save_best_only = False,
                     #mode = 'min',
                     period  = 5
                    ) 
callbacks = [cp]

In [ ]:
model.fit(train_ds,
          batch_size = batch_size, 
          epochs = epochs,
          validation_data = vali_ds,
          shuffle = False,
          callbacks = callbacks)

In [ ]:
model.save('model/PredConvLSTM-final.h5')